# Going Subphraseless

The current method for isolating phrase heads ([here](https://nbviewer.jupyter.org/github/ETCBC/heads/blob/master/phrase_heads.ipynb)) requires strenuous and ineloquent processing of BHSA subphrase relations. The subphrases are not always consistently encoded and suffer from numerous exceptional cases. The result is that the method is rather convoluted and ineloquent.

This notebook will explore the possibility of disconnecting semantic head analysis from the ETCBC subphrase encoding. 

A "semantic" head is the primary content word of a phrase, following Croft's "Primary Information Bearing Unit":

> **The noun and the verb are the PRIMARY INFORMATION_BEARING UNITS (PIBUs) of the phrase and clause respectively. In common parlance, they are the content words. PIBUs have major informational content that functional elements such as articles and [auxiliaries] do not have. (Croft, *Radical Construction Grammar*, 2001, 258; see also Shead, *Radical Frame Semantics and Biblical Hebrew*, 104)**

> **A (semantic) head is the profile equivalent that is the primary information-bearing unit, that is, the most contentful item that most closely profiles the same kind of thing that the whole constituent profiles. (ibid., 259)**

Croft also provides an additional criterion to "profile equivalence":

> **If the criterion of profile equivalence produces two candidates for headhood, the less schematic meaning is the PIBU; that is, the PIBU is the one with the narrower extension, in the formal semantic sense of that term (ibid., 259)**

## Inquiry

Can we isolate semantic phrase heads in BHSA using only the phrase_atom and phrase limits? This question indeed means that we  take the phrase_atom/phrase boundaries for granted. Empirically, the validity of BHSA phrase boundaries needs to be tested. But for now, the exercise of isolating semantic phrase heads could be seen as the first step towards reproducible phrase boundaries.

## Basic Concepts

A semantic head will most often stand in a syntactically independent position. For Hebrew nominal phrases, that essentially means a word which is not precided by a construct, and which is semantically central (excluding attributive slots (e.g. H + noun + H + ATTRIBUTIVE) or an adjectival slots (e.g. noun + noun as in אישׁ טוב).

Quantifier expressions present unique cases, which may be syntactically independent but semantically secondary. These are expressed through specialized lexical items such as cardinal numbers and qualitative quantifiers (e.g.  "כל" and "חצי").

Another complication is the use of nouns as prepositional items. Such uses can be seen with words like פני "face" such as לפני "in front," and even words like ראשׁ as in ראשׁ החדשׁ "beginning of the month." 

Other expressions of quantity, quality, and function provide similar complexities. These cases have to be specified in advance.

### Ambiguity

Considerable ambiguity is present in several of cases:

**`A B and C`**<br>
Given A, B, C == nominal words. Is their relationship `A // B // C` or `A+B // C`. In other words: **what is the relationship of two adjacent nominal words given a list?** Is B a descriptor of A or is it an independent element? 

**`A of B and C`**<br>
Is it, `(A of B) // (C)` or `(A of (B // C)`

Or even:

**`A of B C and D`**<br>
This pattern combines elements from both ambiguous cases.

To address these ambiguities we will apply a battery of disambiguation attempts. Some of those attempts will draw from corpus data, i.e. do we ever see `B and C` with the conjunction explicitly elsewhere in the corpus? Or do we ever see a `A of C` excplicitly in the corpus? Accents may also play a role: do we see a conjunctive or disjunctive accent between `B C`? 

## Prerequisites

A number of pre-defined word sets are needed for processing quantification and ambiguous adjacency. These sets are made available in the form of `wsets`, a dictionary containing word sets that are calculated in to the `wordsets` directory of this repository. The following wordsets have been defined:

* nominals – a set of word nodes with parts of speech and participles that have the potential to function as nominalized elements. The selected parts of speech are quite permissive: `{'subs', 'nmpr', 'adjv', 'advb', 'prde', 'prps', 'prin', 'inrg'}`. Since parts of speech are not taken as universal linguistic categories but only summaries of language-specific word tendencies (cf. Croft, *Radical Construction Grammar*, 2001), we consider that almost any part of speech can be used in a nominal pattern (or construction). There are some upper limits to this assumption, though. For instance, we exclude cojunctions, articles, prepositions, and negators. 
* prepositions – a word set consisting of words with a part of speech category of `prep`, a lexical set (`ls`) feature of `ppre` ("potential preposition"), as well as a select group of nouns like פני "face" which have been processed for prepositionality. 
* quantifiers - consists of word nodes that are cardinal numbers or qualitative quantifiers such as כל.
* mword – mapping from a word to its phonological word group ("masoretic word"); joins words on maqqeph and ø space
* accent_type – a mapping from a word to its accent type: conjunctive or disjunctive
* conj_pairs – a dict of observed conjunction pairings of lexemes in the corpus: `A & B`
* cons_pairs – a dict of observed construct pairings of lexemes in the corpus: `A of B`
* mom – mapping from word node to its mother word node for a specified relationship: `mom[A]['coord'] = B`
* kid – opposite of mom; mapping from word to its children nodes for a relationship: `kid[A]['cons'] = B`

**Let's get started**. We load the necessary functions and BHSA data (straight from source).

In [1]:
import collections
import pickle
import random
import re
from IPython.display import display, HTML
from datetime import datetime
from pprint import pprint
from tf.app import use
wsets = pickle.load(open('wordsets/wsets.pickle', 'rb'))
A = use('bhsa', hoist=globals(), silent=True)

   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


### Wordsets

In [2]:
wsets.keys()

dict_keys(['noms', 'preps', 'quants', 'accent_type', 'mwords', 'conj_pairs', 'cons_pairs'])

In [3]:
list(wsets['cons_pairs'].keys())[:10]

['>JC/', 'KL/', 'BN/', 'TPF[', 'MZBX/', 'B>R/', '<T/', 'XRB/', 'RWX/', 'MLK/']

In [7]:
#wsets['conj_pairs']['>JC/']

# Machinery

We could use some machinery to do the hard work of looking in and around a node. In the older approach we used TF search templates. But these are not very efficient at scale, and they are always bound by the limits of the query language. I take another approach here: a class which, when called, can give all sorts of contextual information about any node which is fed in. This class is related to the `Mom` class used in wordsets ([here](wordsets/context.py)), and in fact it depends on the output of that class which is stored in the `mom` and `kid` word sets.

The "machines" we use are defined in the positions module. They are series of position testers for a node and string evaluators. By writing our conditions as strings, we can maintain a mapping between the truth conditions and their results. This is intended to aid debugging and the process of elimination which we apply herein.

In [8]:
from wordsets.positions import Getter, Positions, Evaluator, showconds

### Search Cache For Keeping Queries

In [10]:
run = input('refresh cache?')

if run == 'y':
    cache = []
else:
    pass

refresh cache? n


## Grammar

This class contains patterns needed to classify constructions in Hebrew phrases.

In [79]:
class Grammar:
    
    def __init__(self, wsets, tf):
        self.tf = tf
        self.wsets = wsets
        self.tests = {
            'cons': self.cons,
            'adjv': self.adjv,
            'disj': self.disj,
        }
        # cache results for recursive calls?
        self.cache = collections.defaultdict(lambda:collections.defaultdict())
        
        
    def getP(self, w):
        return Positions(w, 'phrase_atom', self.tf).get
    
    def test(self, condtuple):
        test = [ruleset for ruleset in condtuple
                    if all(ruleset['cnd'].values())
               ]
        
        # NB rules are written so as to default
        # on the last-most matching item
        # this allows for more complex situations
        # to default if true
        if test:
            return test[-1]
        else:
            return {}
        
    def cons(self, w):
        
        '''
        Queries for construct relations
        on a word.
        '''
        
        params = {'match': None, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        
        params['conds'] = (

            {
                'name': 'cons',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {

                    'P(-1, st) == c': 
                        P(-1,'st') == 'c',

                    'P(-1) not in quants|preps': 
                        P(-1) not in quants|preps,
                }
            },

            {
                'name': 'cons',
                'src': P(0),
                'tgt': P(-2),
                'cnd': {

                    'P(-1,sp) == art': 
                        P(-1,'sp') == 'art',

                    'P(-2, st) == c': 
                        P(-2,'st') == 'c',

                    'P(-2) not in quants|preps': 
                        P(-2) not in quants|preps,
                }
            }
        )

        params['match'] = self.test(params['conds'])
        return params

    def adjv(self, w):
        '''
        Searches for adjective relas on a word.
        '''
                
        params = {'match': None, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        
        # check for recursive adjective matches 
        a2match = self.adjv(P(-1))
                
        params['conds'] = (

            {
                'name': 'adjv',
                'src': P(0),
                'tgt': P(1),
                'cnd': {

                    'P(0,sp) == advb':
                        P(0,'sp') == 'advb',

                    'P(1) in noms':
                        P(1) in noms,
                }
            },
            
            {
                'name': 'adjv',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {

                    'P(0, sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},

                    'P(-1, st) & {NA, a}': 
                        P(-1,'st') in {'NA', 'a'},

                    'P(-1) in noms':
                        P(-1) in noms,

                    'P(-1) not in {quants|preps}':
                        P(-1) not in quants|preps,

                }
            },
            
            {
                'name': 'adjv2x',
                'src': P(0),
                'tgt': a2match.get('tgt', None),
                'cnd': {
                    
                    'P(0,sp) in {adjv, verb}':
                        P(0,'sp') in {'adjv', 'verb'},
                    
                    'P(-1,st) & {NA, a}': 
                        P(-1,'st') in {'NA', 'a'},
                    
                    'P(-1) in noms':
                        P(-1) in noms,
                    
                    'P(-1) not in {quants|preps}':
                        P(-1) not in quants|preps,
                    
                     'self.adjv(P(-1)) and target != P(0)':
                        bool(a2match['match']) and a2match.get('tgt', None) != P(0)
                }
            },

        )

        params['match'] = self.test(params['conds'])
        return params
        
    def disj(self, w):
        
        '''
        Queries for disjunct relations,
        i.e. two adjacent nominals that are
        not related.
        '''
        
        params = {'match': None, 'conds': None}
        
        if not w:
            return params
        
        P = self.getP(w)
        wsets = self.wsets
        quants, preps, noms = wsets['quants'], wsets['preps'], wsets['noms']
        conjpairs = wsets['conj_pairs']
        atype = wsets['accent_type']
        mword = wsets['mwords']
        
        params['conds'] = (

            {
                'name': 'disjoint',
                'src': P(0),
                'tgt': P(-1),
                'cnd': {
                    
                    'P(0) in noms':
                        P(0) in noms,
                    
                    'not {P(0), P(-1)} & quants':
                        not {P(0), P(-1)} & quants,
                    
                    'P(-1) in noms':
                        P(-1) in noms,
                    
                    'P(-1, st) == a': 
                        P(-1,'st') == 'a',
                    
                    'P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct':
                        (
                            P(-1,'lex') in conjpairs[P(0,'lex')]
                            or atype.get(P(-1)) == 'disjunct' and P(-1) not in mword[w]
                        ),
                    
                    'not self.adjv(w)[match]':
                        not self.adjv(w)['match']
                }
            },
        )

        params['match'] = self.test(params['conds'])
        return params

<hr>

In [80]:
def prettyconds(rulesets):
    '''
    Iterate through an explain dict for a rela
    and print out all of checked conditions.
    '''
    
    for ruleset in rulesets:
        src, tgt = ruleset['src'], ruleset['tgt']
        print(f'{src} -> {tgt}')
        
        for cond, value in ruleset['cnd'].items():
            print('{:<30} {:>30}'.format(cond, str(value)))
        
        print()
        
def showmatch(match):
    
    '''
    Displays a match from a Grammar test.
    '''
    
    hit, conds = match['match'], match['conds']
    
    if not hit:
        print('NO MATCHES')
        print('-'*20)
        A.pretty(L.u(conds[0]['src'], 'phrase_atom')[0], extraFeatures='sp st', withNodes=True)
        prettyconds(conds)
        return None

    name, src, tgt = hit['name'], hit['src'], hit['tgt']

    phrase = L.u(src, 'phrase_atom')[0]

    highlights = {src:'pink',
                  tgt:'lightgreen'}

    A.pretty(phrase, withNodes=True, extraFeatures='sp st', highlights=highlights)
    prettyconds(conds)
    display(HTML('<hr>'))
        
def test_search(relastr, show=10, end=None, name=''):
    '''
    Searches phrases with the specified relation 
    and prints out their descriptive explanation.
    '''
    
    start = datetime.now()
    print('beginning search')
    
    # build a convenient test set of words
    phrases = [ph for ph in F.otype.s('phrase_atom') 
                   if F.typ.v(ph) in {'NP', 'PP'}
                   and len(L.d(ph, 'word')) > 1
              ]
    words = [w for ph in phrases for w in L.d(ph, 'word')
                if w in wsets['noms']
                and F.language.v(w) == 'Hebrew'
            ]
    
    # random shuffle to get good diversity of examples
    random.shuffle(words)
    
    # set up grammar
    G = Grammar(wsets, A)
    
    matches = []
    append = matches.append
    
    # iterate and find matches on words
    for i,w in enumerate(words):

        # update every 5000 iterations
        if i%5000 == 0:
            print(f'\t{len(matches)} found ({i}/{len(words)})')
        
        # run grammar search
        test = G.tests[relastr](w)
        
        # save results
        if test['match']:
            if not name:
                append(test)
            elif test['match']['name'] == name:
                append(test)
            
        # stop at end
        if len(matches) == end:
            break
        
        
    # display
    print('done at', datetime.now() - start)
    print(len(matches), 'matches found...')
    print('showing', end)
    
    for match in matches[:show]:
        showmatch(match)
        
    cache.append(matches)

<hr>

In [81]:
G = Grammar(wsets, A)

In [82]:
showmatch(G.tests['disj'](111137))

NO MATCHES
--------------------


111137 -> 111136
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                 False



In [83]:
showmatch(G.tests['adjv'](19986))

NO MATCHES
--------------------


19986 -> 19987
P(0,sp) == advb                                         False
P(1) in noms                                             True

19986 -> 19985
P(0, sp) in {adjv, verb}                                False
P(-1, st) & {NA, a}                                      True
P(-1) in noms                                            True
P(-1) not in {quants|preps}                              True

19986 -> None
P(0,sp) in {adjv, verb}                                 False
P(-1,st) & {NA, a}                                       True
P(-1) in noms                                            True
P(-1) not in {quants|preps}                              True
self.adjv(P(-1)) and target != P(0)                          False



!! ADD A NAME CONSTRUCTION: SUBS + NMPR

<hr>

In [84]:
test_search('disj', name='', show=100, end=10)

beginning search
	0 found (0/121804)
done at 0:00:23.786448
10 matches found...
showing 10


88162 -> 88161
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



198043 -> 198042
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



154424 -> 154423
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



249329 -> 249328
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



...

45410 -> 45409
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



129491 -> 129490
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



241672 -> 241671
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



308688 -> 308687
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



43842 -> 43841
P(0) in noms                                             True
not {P(0), P(-1)} & quants                               True
P(-1) in noms                                            True
P(-1, st) == a                                           True
P(-1,lex) in conjpairs[P(0,lex)] or P(-1) == disjunct                           True
not self.adjv(w)[match]                                  True



<br>
<br>
<br>
<br>
<br>
<br>
<br>
<hr>

In [95]:
# A.table(A.search('''

# word st=a

# '''), end=5)

### Testing

This machinery will allow us to write large yet concise conditional statements that test all kinds of parameters around the context.

Let's make a set of all NPs in the corpus from which we can gradually work from. We will work with phrase_atoms for now.

In [19]:
nps = set(F.typ.s('NP')) & set(F.otype.s('phrase_atom'))
covered = set()

def prog():
    # print remaining cases
    print(len(nps)-len(covered))

print(len(nps))

47504


Let's eliminate all options that have no other choices but a candidate.

In [20]:
simpleres = []

for p in nps:
    cands = [w for w in L.d(p, 'word') if F.sp.v(w) in cand_sps]
    if len(cands) == 1:
        simpleres.append((p, cands[0]))
        
len(simpleres)

27766

In [9]:
def get_quantified(word, tf, **wsets):
    '''
    Recursively calls down a quantifier chain until
    finding a quantified word.
    '''
    
    quants, noms = wsets['quants'], wsets['noms']

    P = Positions(word, 'phrase_atom', tf).get

    target = (
        lambda n: P(n), 
        lambda n: P(n) not in quants,
        lambda n: P(n, 'sp') in noms,
    )

    # check this word
    if all(cond(0) for cond in target):
        return word
    
    # check next word
    elif all(cond(P(1)) for cond in target):
        return P(1)
    
    # move up one
    else:
        return get_quantified(P(1))